In [1]:
from pandas.io.sql import read_sql
from pyiem.util import get_dbconn

COOP = get_dbconn("coop")

df = read_sql(
    """
WITH data as (
    SELECT station, year, min(extract(doy from day)) as doy from alldata
    WHERE low < 32 and month > 6 and substr(station, 3, 1) != 'C' and
    substr(station, 3, 4) != '0000' GROUP by station, year),
agg1 as (
   SELECT station, avg(doy), max(year) from data GROUP by station
)
   
SELECT agg1.station, agg1.avg, ST_x(geom), ST_Y(geom), agg1.max, t.state from stations t, agg1
WHERE agg1.station = t.id and t.network ~* 'CLIMATE' and
t.state in ('IA', 'ND', 'SD', 'KS', 'NE', 'MO', 'MN', 'WI', 'IL', 'IN', 'OH', 'KY', 'MI')
""",
    COOP,
    index_col="station",
)

In [2]:
df[(df["state"] == "KS") & (df["max"] == 2020)]

,avg,st_x,st_y,max,state
station,,,,,
KS8235,281.078125,-101.766667,38.450000,2020,KS
KS1029,279.975309,-101.433333,39.366667,2020,KS


In [6]:
import datetime

labels = []
ticks = []
tickrange = range(int(df["avg"].min()) - 2, int(df["avg"].max()) + 2, 5)
for i in tickrange:
    ticks.append(i)
    ts = datetime.datetime(2000, 1, 1) + datetime.timedelta(days=(i - 1))
    labels.append(ts.strftime("%-d %b"))

In [9]:
df["x"] = "x"
df2 = df[df["max"] == 2020]
from pyiem.plot import MapPlot, get_cmap

m = MapPlot(
    sector="midwest",
    subtitle='Climatology based on long term climate sites, "x" denotes 2020 observation below 32 till 24 Sep',
    title="Average First Fall Date below 32$^\circ$F Air Temperature",
)
cmap = get_cmap("Spectral")
m.contourf(
    df["st_x"],
    df["st_y"],
    df["avg"],
    tickrange,
    clevlabels=labels,
    cmap=cmap,
    extend="neither",
)
m.plot_values(df2["st_x"], df2["st_y"], df2["x"], "%s", labelbuffer=3)
# m.drawcounties()

m.postprocess(filename="200925.png")